In [1]:
from datasets import load_dataset
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score

In [2]:
dataset = load_dataset('cardiffnlp/tweet_topic_single')

Found cached dataset tweet_topic_single (C:/Users/schmi/.cache/huggingface/datasets/cardiffnlp___tweet_topic_single/tweet_topic_single/1.0.4/832eaa087889d9f4bc549869b44e0acb85a78364dfb3d2bc0bdf23a7224cf2ce)


  0%|          | 0/13 [00:00<?, ?it/s]

In [3]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(dataset["train_all"]["text"]) # type: ignore

X_test_counts = count_vect.transform(dataset["test_2021"]["text"]) # type: ignore

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [4]:
clf = MultinomialNB().fit(X_train_counts, dataset["train_all"]["label"]) # type: ignore

predicted = clf.predict(X_test_counts)

print(np.mean(predicted == dataset["test_2021"]["label"])) # type: ignore

print(f1_score(dataset["test_2021"]["label"], predicted, average='macro')) # type: ignore



0.7542823390431187
0.40392919454434506


In [5]:
clf = MultinomialNB().fit(X_train_tfidf, dataset["train_all"]["label"]) # type: ignore

predicted = clf.predict(X_test_counts)

print(np.mean(predicted == dataset["test_2021"]["label"])) # type: ignore

print(f1_score(dataset["test_2021"]["label"], predicted, average='macro')) # type: ignore

0.7028942705256941
0.2754189380431041


In [6]:
count_vect = CountVectorizer(binary=True)
X_train_counts_b = count_vect.fit_transform(dataset["train_all"]["text"]) # type: ignore

X_test_counts_b = count_vect.transform(dataset["test_2021"]["text"]) # type: ignore

tfidf_transformer = TfidfTransformer()
X_train_tfidf_b = tfidf_transformer.fit_transform(X_train_counts_b)

In [7]:
clf = MultinomialNB().fit(X_train_counts_b, dataset["train_all"]["label"]) # type: ignore

predicted = clf.predict(X_test_counts_b)

print(np.mean(predicted == dataset["test_2021"]["label"])) # type: ignore

print(f1_score(dataset["test_2021"]["label"], predicted, average='macro')) # type: ignore


0.7442409923213231
0.376958701895147


In [8]:
clf = MultinomialNB().fit(X_train_tfidf_b, dataset["train_all"]["label"]) # type: ignore

predicted = clf.predict(X_test_counts_b)

print(np.mean(predicted == dataset["test_2021"]["label"])) # type: ignore

print(f1_score(dataset["test_2021"]["label"], predicted, average='macro')) # type: ignore

0.7040756054341406
0.2747519063565876


In [17]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
docs = dataset["train_all"]["text"] # type: ignore	

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\schmi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\schmi\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [18]:
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

In [19]:
# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

In [20]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs) # type: ignore

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.7)

2023-06-30 16:40:16,260 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2023-06-30 16:40:16,397 : INFO : built Dictionary<15550 unique tokens: ['bb', 'beat', 'cedar', 'clinton', 'division']...> from 4374 documents (total 111634 corpus positions)
2023-06-30 16:40:16,398 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<15550 unique tokens: ['bb', 'beat', 'cedar', 'clinton', 'division']...> from 4374 documents (total 111634 corpus positions)", 'datetime': '2023-06-30T16:40:16.398410', 'gensim': '4.3.1', 'python': '3.11.0 | packaged by Anaconda, Inc. | (main, Mar  1 2023, 18:18:21) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'created'}
2023-06-30 16:40:16,414 : INFO : discarding 14883 tokens: [('bb', 5), ('cedar', 1), ('clinton', 2), ('division', 7), ('edward', 12), ('evan', 5), ('hr', 6), ('ip', 3), ('josh', 6), ('kernel', 1)]...
2023-06-30 16:40:16,415 : INFO : keeping 667 tokens which were in no less than 20 and no more tha

In [21]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 667
Number of documents: 4374


In [22]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 6
chunksize = 2000
passes = 20 ##################################################20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=1
)

2023-06-30 16:40:18,700 : INFO : using autotuned alpha, starting with [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667]
2023-06-30 16:40:18,701 : INFO : using serial LDA version on this node
2023-06-30 16:40:18,702 : INFO : running online (multi-pass) LDA training, 6 topics, 20 passes over the supplied corpus of 4374 documents, updating model once every 2000 documents, evaluating perplexity every 2000 documents, iterating 400x with a convergence threshold of 0.001000
2023-06-30 16:40:20,400 : INFO : -7.229 per-word bound, 150.1 perplexity estimate based on a held-out corpus of 2000 documents with 34571 words
2023-06-30 16:40:20,401 : INFO : PROGRESS: pass 0, at document #2000/4374
2023-06-30 16:40:21,786 : INFO : optimized alpha [0.09593601, 0.078850694, 0.119479045, 0.10611991, 0.119313434, 0.09597969]
2023-06-30 16:40:21,788 : INFO : merging changes from 2000 documents into a model of 4374 documents
2023-06-30 16:40:21,789 : INFO : topic #1 (0.079): 0.040*"to" 

In [23]:
top_topics = model.top_topics(corpus)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

2023-06-30 16:41:23,525 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2023-06-30 16:41:23,534 : INFO : CorpusAccumulator accumulated stats from 2000 documents


2023-06-30 16:41:23,543 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2023-06-30 16:41:23,550 : INFO : CorpusAccumulator accumulated stats from 4000 documents


Average topic coherence: -2.6771.
[([(0.06828979, 'the'),
   (0.04007417, 'to'),
   (0.032969132, 'and'),
   (0.026212731, 'it'),
   (0.023183223, 'you'),
   (0.023172032, 'of'),
   (0.02107132, 'is'),
   (0.019993745, 'in'),
   (0.018546622, 'that'),
   (0.013927067, 'for'),
   (0.012777872, 'on'),
   (0.0124244485, 'so'),
   (0.012367796, 'this'),
   (0.011975727, 'be'),
   (0.011713104, 'my'),
   (0.011351575, 'username'),
   (0.010690456, 'have'),
   (0.0104199005, 'but'),
   (0.010012997, 'wa'),
   (0.009609008, 'are')],
  -1.7636969170959949),
 ([(0.14865837, 'the'),
   (0.050201427, 'of'),
   (0.03812995, 'in'),
   (0.032766238, 'username'),
   (0.02675149, 'to'),
   (0.025624294, 'and'),
   (0.021888444, 'for'),
   (0.019953173, 'url'),
   (0.018517429, 'on'),
   (0.017370617, 'game'),
   (0.016746443, 'is'),
   (0.0132305585, 'at'),
   (0.0127580715, 'with'),
   (0.011137574, 'this'),
   (0.009250581, 'team'),
   (0.008817519, 'from'),
   (0.0077880668, 'by'),
   (0.00689963, 

In [24]:
from sklearn.metrics import accuracy_score
docs_test2021_label = dataset["test_2021"]["label"]


docs_test2021_text = dataset["test_2021"]["text"]
# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs_test2021_label)):
    docs_test2021_text[idx] = docs_test2021_text[idx].lower()  # Convert to lowercase.
    docs_test2021_text[idx] = tokenizer.tokenize(docs_test2021_text[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs_test2021_text = [[token for token in doc if not token.isnumeric()] for doc in docs_test2021_text]
# Remove words that are only one character.
docs_test2021_text = [[token for token in doc if len(token) > 1] for doc in docs_test2021_text]

lemmatizer = WordNetLemmatizer()
docs_test2021_text = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs_test2021_text]

# Prepare the testing data
test_corpus = [dictionary.doc2bow(doc) for doc in docs_test2021_text]

# Infer topic distributions for the test documents
test_topic_distributions = [model[doc] for doc in test_corpus]

# Convert topic distributions into labels
predicted_labels = [max(doc, key=lambda x: x[1])[0] for doc in test_topic_distributions]

# Calculate accuracy
accuracy = accuracy_score(docs_test2021_label, predicted_labels)
print("Accuracy:", accuracy)

print("f1_score:", f1_score(docs_test2021_label, predicted_labels, average='macro')) # type: ignore

Accuracy: 0.3603071470761961
f1_score: 0.18015009515897837
